In [7]:
! pip install boto3
import getpass

     |████████████████████████████████| 143kB 6.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 18.6MB/s eta 0:00:01
     |████████████████████████████████| 11.2MB 18.4MB/s eta 0:00:01
     |████████████████████████████████| 256kB 107.8MB/s eta 0:00:01
     |████████████████████████████████| 143kB 82.3MB/s eta 0:00:01
ERROR: refractml 1.0.3 has requirement PyYAML==6.0, but you'll have pyyaml 6.0.1 which is incompatible.
ERROR: refractml 1.0.3 has requirement urllib3==1.26.15, but you'll have urllib3 1.26.16 which is incompatible.
ERROR: jupyterlab-server 2.23.0 has requirement jsonschema>=4.17.3, but you'll have jsonschema 3.2.0 which is incompatible.


In [1]:
access_id = getpass.getpass("Enter aws access ID : ")
access_key = getpass.getpass("Enter aws access KEY : ")

Enter aws access ID : ········
Enter aws access KEY : ········


In [2]:
import boto3

s3 = boto3.client('s3',
         aws_access_key_id=access_id,
         aws_secret_access_key= access_key)

In [3]:
bucket_name = "sagemaker-us-east-1-123884475188"
object_name = "huggingface-pytorch-training-2023-09-13-07-45-20-615/output/model.tar.gz"
file_name = "/data/sagemaker/model.tar.gz"

In [4]:
s3.download_file(bucket_name, object_name, file_name)

In [9]:
! ls /data/sagemaker

HBS  model.tar.gz


In [10]:
import tarfile
with tarfile.open("/data/sagemaker/model.tar.gz", 'r') as tar:
    tar.extractall("/data/sagemaker/HBS")

In [11]:
! ls /data/sagemaker/HBS

checkpoint-500	   special_tokens_map.json  training_args.bin
config.json	   tokenizer.json	    vocab.txt
pytorch_model.bin  tokenizer_config.json


In [12]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("/data/sagemaker/HBS")
tokenizer = AutoTokenizer.from_pretrained("/data/sagemaker/HBS")

In [13]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."
encoded_input = tokenizer(text, return_tensors='pt')
logits = model(**encoded_input).logits

In [14]:
predicted_class_id = logits.argmax().item()

In [15]:
model.config.id2label[predicted_class_id]

'LABEL_1'